In [55]:
Пайплайн
========
Часть 3.

1. Смотрит пути
2. Готовит DataFrame из photos.csv
3. Генерирует geojson с полями из DataFrame - 
4. Добавляет код страны через "https://nominatim.openstreetmap.org/reverse"

SyntaxError: invalid syntax (946500708.py, line 2)

In [26]:
import pandas as pd
import json
import os

# === 📁 Пути к файлам
CSV_PATH = "csv/photos.csv"
GEOJSON_PATH = "geojson/photos.geojson"

# === 🗂️ Проверяем, существует ли папка geojson
if not os.path.exists('geojson'):
    os.makedirs('geojson')

In [28]:
# === 📦 Читаем CSV
df = pd.read_csv(CSV_PATH)

# === 🎯 Фильтруем данные с координатами
df = df.dropna(subset=['latitude', 'longitude'])

# === 🗺️ Генерация GeoJSON
features = []
for _, row in df.iterrows():
    features.append({
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row['longitude'], row['latitude']]
        },
        "properties": {
            "filename": row['filename'],
            "folder": row['folder'],
            "year": row['year'],
            "month": row['month'],
            "day": row['day'],
            "source_type": row['source_type']
        }
    })

In [30]:
# === 💾 Сохраняем в файл
with open(GEOJSON_PATH, 'w', encoding='utf-8') as f:
    json.dump({
        "type": "FeatureCollection",
        "features": features
    }, f, ensure_ascii=False, indent=4)

print(f"✅ GeoJSON успешно сгенерирован: {GEOJSON_PATH}")
print(f"📌 Всего точек добавлено: {len(features)}")

✅ GeoJSON успешно сгенерирован: geojson/photos.geojson
📌 Всего точек добавлено: 2065


In [ ]:
# --- если нужно добавить в json поле с кодом страны

In [40]:
import json
import requests
import time
from tqdm import tqdm
import sys

GEOJSON_FILE = "geojson/photos.geojson"
NOMINATIM_URL = "https://nominatim.openstreetmap.org/reverse"
DELAY = 2  # Задержка между запросами, чтобы избежать лимита API

HEADERS = {
    'User-Agent': 'PhotoMapsApp (example@domain.com)'
}

def get_country_code(lat, lon):
    """
    Получение кода страны по координатам через Nominatim API
    """
    try:
        response = requests.get(NOMINATIM_URL, params={
            'format': 'json',
            'lat': lat,
            'lon': lon
        }, headers=HEADERS)
        
        if response.status_code == 200:
            data = response.json()
            country_code = data.get('address', {}).get('country_code', '').upper()
            if country_code:
                return country_code
            else:
                print(f"❌ Ошибка: Страна не найдена для координат {lat}, {lon}")
                sys.exit(1)
        else:
            print(f"❌ Ошибка запроса: статус {response.status_code}")
            sys.exit(1)

    except Exception as e:
        print(f"❌ Ошибка соединения: {e}")
        sys.exit(1)


def enrich_geojson():
    """
    Обогащение GeoJSON данными о стране по координатам
    """
    with open(GEOJSON_FILE, 'r') as file:
        geojson_data = json.load(file)

    # Прогресс-бар
    for feature in tqdm(geojson_data["features"], desc="Обработка точек"):
        lat, lon = feature["geometry"]["coordinates"][1], feature["geometry"]["coordinates"][0]
        country_code = get_country_code(lat, lon)
        feature["properties"]["country_code"] = country_code
        time.sleep(DELAY)  # Пауза, чтобы не словить блокировку API

    with open(GEOJSON_FILE, 'w') as file:
        json.dump(geojson_data, file, indent=4)
        print(f"\n✅ Файл {GEOJSON_FILE} успешно обновлён с кодами стран.")


# === Запуск процесса ===
enrich_geojson()

Обработка точек: 100%|██████████| 2065/2065 [1:20:08<00:00,  2.33s/it]


✅ Файл geojson/photos.geojson успешно обновлён с кодами стран.
